In [2]:
import pandas as pd

df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
cols_with_invalid_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

In [4]:
for col in cols_with_invalid_zeros:
    print(f"{col} 컬럼에서 0의 개수: {(df[col] == 0).sum()}")

Glucose 컬럼에서 0의 개수: 5
BloodPressure 컬럼에서 0의 개수: 35
SkinThickness 컬럼에서 0의 개수: 227
Insulin 컬럼에서 0의 개수: 374
BMI 컬럼에서 0의 개수: 11


In [5]:
import numpy as np

for col in cols_with_invalid_zeros:
    df[col] = df[col].replace(0, np.nan)  # 0을 NaN으로 바꿈
    median = df[col].median()             # 중앙값 구함
    df[col] = df[col].fillna(median)      # NaN을 중앙값으로 대체

In [6]:
from sklearn.preprocessing import StandardScaler

# target은 제외하고 feature만
X = df.drop('Outcome', axis=1)

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.866045,-0.031990,0.670643,-0.181541,0.166619,0.468492,1.425995
1,-0.844885,-1.205066,-0.528319,-0.012301,-0.181541,-0.852200,-0.365061,-0.190672
2,1.233880,2.016662,-0.693761,-0.012301,-0.181541,-1.332500,0.604397,-0.105584
3,-0.844885,-1.073567,-0.528319,-0.695245,-0.540642,-0.633881,-0.920763,-1.041549
4,-1.141852,0.504422,-2.679076,0.670643,0.316566,1.549303,5.484909,-0.020496


In [9]:
# 1. 결측치 처리
# 2. 정규화
# 3. 분할

X = df.drop('Outcome', axis=1)
y_class = df['Outcome']
y_reg = df['Glucose']

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 분류용 분할
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)

# 회귀용 분할
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. 텐서로 변환
X_train_tensor = torch.tensor(X_train_c, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_c.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_c, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_c.values, dtype=torch.float32).view(-1, 1)

# 2. DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. 모델 정의
class MLPClassifier(nn.Module):
    def __init__(self):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # 이진 분류용
        )
    
    def forward(self, x):
        return self.model(x)

# 4. 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x105a04fd0>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ai_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [11]:
# 5. 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/20, Loss: 0.6954
Epoch 2/20, Loss: 0.6204
Epoch 3/20, Loss: 0.5544
Epoch 4/20, Loss: 0.4907
Epoch 5/20, Loss: 0.4810
Epoch 6/20, Loss: 0.4615
Epoch 7/20, Loss: 0.4708
Epoch 8/20, Loss: 0.4460
Epoch 9/20, Loss: 0.4352
Epoch 10/20, Loss: 0.4370
Epoch 11/20, Loss: 0.4225
Epoch 12/20, Loss: 0.4149
Epoch 13/20, Loss: 0.4180
Epoch 14/20, Loss: 0.4154
Epoch 15/20, Loss: 0.4095
Epoch 16/20, Loss: 0.4038
Epoch 17/20, Loss: 0.4065
Epoch 18/20, Loss: 0.3971
Epoch 19/20, Loss: 0.4080
Epoch 20/20, Loss: 0.3961


In [12]:
# 6. 평가
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        predicted = (outputs.cpu().numpy() > 0.5).astype(int)
        total += y_batch.size(0)
        correct += (predicted == y_batch.numpy()).sum()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 75.97%


In [13]:
# 회귀용 텐서 변환
X_train_tensor = torch.tensor(X_train_r, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_r.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_r, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_r.values, dtype=torch.float32).view(-1, 1)

# DataLoader 구성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [3]:
class MLPRegressor(nn.Module):
    def __init__(self):
        super(MLPRegressor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # 활성화 함수 없음
        )
    
    def forward(self, x):
        return self.model(x)

# 장비 설정 및 학습 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'nn' is not defined

In [15]:
# 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/20, Loss: 15553.9887
Epoch 2/20, Loss: 15352.6904
Epoch 3/20, Loss: 15204.5962
Epoch 4/20, Loss: 14469.6544
Epoch 5/20, Loss: 13816.5209
Epoch 6/20, Loss: 12414.0424
Epoch 7/20, Loss: 10497.8493
Epoch 8/20, Loss: 8437.0992
Epoch 9/20, Loss: 6331.6344
Epoch 10/20, Loss: 4272.0687
Epoch 11/20, Loss: 2702.1924
Epoch 12/20, Loss: 1776.6237
Epoch 13/20, Loss: 1196.3055
Epoch 14/20, Loss: 964.2556
Epoch 15/20, Loss: 824.1014
Epoch 16/20, Loss: 786.0971
Epoch 17/20, Loss: 706.5546
Epoch 18/20, Loss: 650.2680
Epoch 19/20, Loss: 591.0960
Epoch 20/20, Loss: 561.7494


In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 평가
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        predictions.extend(outputs)
        actuals.extend(y_batch.numpy())

# MSE 계산
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
print(f"Test RMSE: {rmse:.2f}")

Test RMSE: 24.25


In [17]:
#CNN 분류
# 분류용 데이터
X_train_tensor = torch.tensor(X_train_c, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
y_train_tensor = torch.tensor(y_train_c.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_c, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test_c.values, dtype=torch.float32).view(-1, 1)

# Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [18]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # flatten
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
# 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6198
Epoch 2, Loss: 0.5412
Epoch 3, Loss: 0.4784
Epoch 4, Loss: 0.4506
Epoch 5, Loss: 0.4372
Epoch 6, Loss: 0.4500
Epoch 7, Loss: 0.4163
Epoch 8, Loss: 0.4380
Epoch 9, Loss: 0.4156
Epoch 10, Loss: 0.4392
Epoch 11, Loss: 0.4104
Epoch 12, Loss: 0.3985
Epoch 13, Loss: 0.3958
Epoch 14, Loss: 0.3964
Epoch 15, Loss: 0.3989
Epoch 16, Loss: 0.3954
Epoch 17, Loss: 0.3803
Epoch 18, Loss: 0.3861
Epoch 19, Loss: 0.3825
Epoch 20, Loss: 0.3902


In [20]:
# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        predicted = (outputs > 0.5).astype(int)
        correct += (predicted == y_batch.numpy()).sum()
        total += y_batch.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 74.68%


In [2]:
# 텐서 변환 + CNN용 형태 (unsqueeze)
X_train_tensor = torch.tensor(X_train_r, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
y_train_tensor = torch.tensor(y_train_r.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_r, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test_r.values, dtype=torch.float32).view(-1, 1)

# DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

NameError: name 'torch' is not defined

In [22]:
class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # 활성화 함수 없음 (회귀)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
# 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 15477.7448
Epoch 2, Loss: 13876.1903
Epoch 3, Loss: 8537.9873
Epoch 4, Loss: 2602.8234
Epoch 5, Loss: 1565.0324
Epoch 6, Loss: 1233.8521
Epoch 7, Loss: 1086.4831
Epoch 8, Loss: 1023.5634
Epoch 9, Loss: 933.7635
Epoch 10, Loss: 860.8677
Epoch 11, Loss: 780.2200
Epoch 12, Loss: 718.4413
Epoch 13, Loss: 662.6441
Epoch 14, Loss: 643.8965
Epoch 15, Loss: 536.3581
Epoch 16, Loss: 500.8459
Epoch 17, Loss: 448.9327
Epoch 18, Loss: 392.8699
Epoch 19, Loss: 359.0199
Epoch 20, Loss: 313.8256


In [1]:
# from sklearn.metrics import mean_squared_error
# import numpy as np

# model.eval()
# predictions = []
# actuals = []

# with torch.no_grad():
#     for X_batch, y_batch in test_loader:
#         X_batch = X_batch.to(device)
#         outputs = model(X_batch).cpu().numpy()
#         predictions.extend(outputs)
#         actuals.extend(y_batch.numpy())

# mse = mean_squared_error(actuals, predictions)
# rmse = np.sqrt(mse)
# print(f"Test RMSE: {rmse:.2f}")

NameError: name 'model' is not defined

In [4]:
import torch.nn as nn

class DiabetesClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(self.relu(self.fc2(x)))
        return self.sigmoid(self.fc3(x))

model_c2 = DiabetesClassifier(X_train_c.shape[1])
criterion_c2 = nn.BCELoss()
optimizer_c2 = torch.optim.Adam(model_c2.parameters(), lr=0.001)

# 학습 루프
for epoch in range(50):
    model_c2.train()
    optimizer_c2.zero_grad()
    output = model_c2(X_train_c_tensor)
    loss = criterion_c2(output, y_train_c_tensor)
    loss.backward()
    optimizer_c2.step()

NameError: name 'X_train_c' is not defined

In [6]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("diabetes.csv")

# 데이터 구조 확인
print("📌 컬럼 목록:")
print(df.columns)

print("\n📌 결측치 유무 확인:")
print(df.isnull().sum())

print("\n📌 요약 통계:")
print(df.describe())

📌 컬럼 목록:
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

📌 결측치 유무 확인:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

📌 요약 통계:
       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.5

In [7]:
import numpy as np

# 0을 결측치로 변환할 컬럼 목록
cols_with_zero_as_null = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# 0을 NaN으로 바꾸고 중앙값으로 대체
for col in cols_with_zero_as_null:
    df[col] = df[col].replace(0, np.nan)
    df[col].fillna(df[col].median(), inplace=True)

# 처리 결과 확인
print(df[cols_with_zero_as_null].describe())

          Glucose  BloodPressure  SkinThickness     Insulin         BMI
count  768.000000     768.000000     768.000000  768.000000  768.000000
mean   121.656250      72.386719      29.108073  140.671875   32.455208
std     30.438286      12.096642       8.791221   86.383060    6.875177
min     44.000000      24.000000       7.000000   14.000000   18.200000
25%     99.750000      64.000000      25.000000  121.500000   27.500000
50%    117.000000      72.000000      29.000000  125.000000   32.300000
75%    140.250000      80.000000      32.000000  127.250000   36.600000
max    199.000000     122.000000      99.000000  846.000000   67.100000


/var/folders/xm/9m5jkr551jxg7scv_lxlytdm0000gn/T/ipykernel_57713/2127489170.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/xm/9m5jkr551jxg7scv_lxlytdm0000gn/T/ipykernel_57713/2127489170.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [8]:
from sklearn.preprocessing import StandardScaler

# 입력 피처(X), 분류용 타겟(y_class), 회귀용 타겟(y_reg)
X = df.drop(columns=['Outcome', 'Glucose'])  # Glucose는 회귀 타겟으로 따로 사용
y_class = df['Outcome']                      # 분류용
y_reg = df['Glucose']                        # 회귀용

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
import torch

# 분류용 데이터셋 분할
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)

# 회귀용 데이터셋 분할
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)

# 분류용 텐서 변환
X_train_c_tensor = torch.tensor(X_train_c, dtype=torch.float32)
y_train_c_tensor = torch.tensor(y_train_c.values, dtype=torch.float32).view(-1, 1)
X_test_c_tensor = torch.tensor(X_test_c, dtype=torch.float32)
y_test_c_tensor = torch.tensor(y_test_c.values, dtype=torch.float32).view(-1, 1)

# 회귀용 텐서 변환
X_train_r_tensor = torch.tensor(X_train_r, dtype=torch.float32)
y_train_r_tensor = torch.tensor(y_train_r.values, dtype=torch.float32).view(-1, 1)
X_test_r_tensor = torch.tensor(X_test_r, dtype=torch.float32)
y_test_r_tensor = torch.tensor(y_test_r.values, dtype=torch.float32).view(-1, 1)

In [10]:
import torch
import torch.nn as nn

# 분류 딥러닝 모델 정의 (클래스 기반, Dropout 포함)
class DiabetesClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(self.relu(self.fc2(x)))
        return self.sigmoid(self.output(x))

# 모델 생성 및 학습 준비
model_c2 = DiabetesClassifier(X_train_c_tensor.shape[1])
criterion_c2 = nn.BCELoss()
optimizer_c2 = torch.optim.Adam(model_c2.parameters(), lr=0.001)

# 학습 루프
for epoch in range(50):
    model_c2.train()
    optimizer_c2.zero_grad()
    output = model_c2(X_train_c_tensor)
    loss = criterion_c2(output, y_train_c_tensor)
    loss.backward()
    optimizer_c2.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 평가
model_c2.eval()
with torch.no_grad():
    preds = model_c2(X_test_c_tensor)
    preds_cls = (preds > 0.5).float()

from sklearn.metrics import classification_report
print(classification_report(y_test_c_tensor, preds_cls))

Epoch 10, Loss: 0.6590
Epoch 20, Loss: 0.6257
Epoch 30, Loss: 0.5953
Epoch 40, Loss: 0.5632
Epoch 50, Loss: 0.5404
              precision    recall  f1-score   support

         0.0       0.73      0.84      0.78        99
         1.0       0.60      0.44      0.51        55

    accuracy                           0.69       154
   macro avg       0.66      0.64      0.64       154
weighted avg       0.68      0.69      0.68       154



In [ ]:
import torch
import torch.nn as nn

# 회귀 모델 정의
class GlucoseRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(torch.relu(self.fc3(x)))
        return self.output(x)  # 회귀이므로 시그모이드 안 씀

# 모델 정의
model_r2 = GlucoseRegressor(X_train_r_tensor.shape[1])
criterion_r2 = nn.MSELoss()
optimizer_r2 = torch.optim.Adam(model_r2.parameters(), lr=0.001)

# 학습 루프
for epoch in range(50):
    model_r2.train()
    optimizer_r2.zero_grad()
    output = model_r2(X_train_r_tensor)
    loss = criterion_r2(output, y_train_r_tensor)
    loss.backward()
    optimizer_r2.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 평가
model_r2.eval()
with torch.no_grad():
    preds_r = model_r2(X_test_r_tensor)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE:", mean_squared_error(y_test_r_tensor, preds_r))
print("R2 Score:", r2_score(y_test_r_tensor, preds_r))